In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import seaborn
from sklearn import svm

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
test_df["employee1"] = train_df["employee1"].astype("str").str.replace(",", "").astype("float")
test_df["employee2"] = train_df["employee2"].astype("str").str.replace(",", "").astype("float")

In [ ]:
train_df["employee1"] = train_df["employee1"].astype("float")
train_df["employee2"] = train_df["employee2"].astype("float")
train_df["OC"] = train_df["OC"].astype("str").str.replace(" ", "")

In [ ]:
train_test_df = train_df.append(test_df)

In [ ]:
train_df

In [ ]:
train_test_df

In [ ]:
factor_columns = train_test_df.select_dtypes(include = ['object']).columns
numeric_columns = train_test_df.columns.difference(factor_columns)

In [ ]:
train_test_df.loc[train_test_df.inst_id == 430, ['instkind']] = 'dental_clinic'
train_test_df.loc[train_test_df.inst_id == 430, ['bedCount']] = 0
train_test_df.loc[train_test_df.inst_id == 413, ['bedCount']] = -999

In [ ]:
train_test_df[factor_columns] = train_test_df[factor_columns].fillna("Not_Sure")

In [ ]:
train_test_df[numeric_columns] = train_test_df[numeric_columns].fillna(-999)

In [ ]:
train_test_df

In [ ]:
fac_le = LabelEncoder()

In [ ]:
train_test_df[factor_columns] = train_test_df.loc[: , factor_columns].apply(lambda x: fac_le.fit_transform(x))

In [ ]:
train_test_df

In [ ]:
train_df = train_test_df.loc[train_test_df["OC"] != 0, :]
test_df = train_test_df.loc[train_test_df["OC"] == 0, :]
train_df["OC"] = train_df["OC"] - 1

In [ ]:
train_df

In [ ]:
sub_id = test_df.inst_id
dep = ['OC']
indep = train_test_df.columns.difference(dep)

In [ ]:
train_X = train_df[indep]
train_Y = train_df[dep]
test_X = test_df[indep]

In [ ]:
np.random.seed(100)
RF = RandomForestClassifier(n_estimators=40, max_features=20)
RF_model = RF.fit(train_X, train_Y)
RF_prd = RF.predict(test_X)
RF_prd_prob = RF.predict_proba(test_X)

In [ ]:
RF_prd

In [ ]:
submission = pd.DataFrame(sub_id)

In [ ]:
submission["OC"] = RF_prd

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
# RF_model.feature_importances_
test_df.columns

In [ ]:
plt.figure(figsize=(40,20))
seaborn.barplot(RF_model.feature_importances_, test_X.columns)

In [ ]:
print('feature 들의 평균의 값')
print(test_df.mean())
print('\nfeature들의  분산 값')
print(test_df.var())

In [ ]:
columns_weights = list(zip(test_X.columns, RF_model.feature_importances_))
columns_weights.sort(key = lambda x : x[1], reverse=True)
# columns_weights
svm_columns = [(columns_weights[x])[0] for x in range(3)]
svm_columns.remove('inst_id')
svm_columns

In [ ]:
svm_train_x = train_X[svm_columns]
svm_train_y = train_Y.OC
svm_test_x = test_X[svm_columns]
svm_train_y

In [ ]:
svm_clf = svm.SVC()
# svm_clf.fit(svm_train_x, svm_train_y)
svm_clf.fit(train_X, train_Y.OC)

In [ ]:
# svm_clf.predict(svm_train_x)
svm_clf.predict(test_X)
# len(test_X)